<a href="https://colab.research.google.com/github/nebojsa-bozanic/OSuM/blob/master/OSuM_Vezba8/OSuM_Vezba8/OSuM_vezba_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Obrada slike u medicini (BMI)</h1>

Literatura: https://moodle.telekom.ftn.uns.ac.rs/course/view.php?id=82


Laboratorijska vežba 8: Obrada rendgenske slike: Suzbijanje šuma 2

Potrebne biblioteke: cv2, numpy, matplotlib.pyplot, pywt, bm3d, os, scipy. ndimage.filters (convolve1d), osum

In [ ]:
!git clone https://github.com/nebojsa-bozanic/OSuM.git
%cd /content/OSuM/OSuM_Vezba8/OSuM_Vezba8

fatal: destination path 'OSuM' already exists and is not an empty directory.
/content/OSuM/OSuM_Vezba8/OSuM_Vezba8
